In [32]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from google.cloud import bigquery
import google.auth

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from detectdd.config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
def get_client(c,p):
    return bigquery.Client(p, c)

def get_gcreds(scopes = None):
    if scopes is None:
        scopes = ["https://www.googleapis.com/auth/bigquery"]
    return google.auth.default(
        scopes )

def get_authed_client():
    cred = get_gcreds()
    print(*cred)
    return get_client(*cred)

def main():
    mimic_drugs = f"""
        SELECT itemid, label, abbreviation, category, unitname
        FROM {icu_d_items}
        WHERE linksto='inputevents' """

    mimic_job = get_authed_client().query(mimic_drugs)
    df = mimic_job.to_dataframe()
    df.label = df.label.fillna("")
    df.label = df.label.apply(str.lower)
    return df

icu_drugs = main()

<google.oauth2.credentials.Credentials object at 0x0000022806B19CD0> mimic-iv-desktop


In [34]:
def read_drugs():
    df = pd.read_csv(data_dir / "NDC_product_table.csv", encoding='ISO-8859-1')
    df.NONPROPRIETARYNAME = df.NONPROPRIETARYNAME.fillna("")
    df.NONPROPRIETARYNAME = df.NONPROPRIETARYNAME.apply(str.lower)
    df.PROPRIETARYNAME = df.PROPRIETARYNAME.fillna("")
    df.PROPRIETARYNAME = df.PROPRIETARYNAME.apply(str.lower)
    return df
ndc_drug_synonyms = read_drugs()
ndc_drug_synonyms

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800_662164fd-5ea0-4a08-bfd1-6b08bdd73342,0002-0800,HUMAN OTC DRUG,sterile diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,BLA,BLA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20221231.0
1,0002-1200_480fceef-6596-4478-97de-677c155506b3,0002-1200,HUMAN PRESCRIPTION DRUG,amyvid,NaN,florbetapir f 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20221231.0
2,0002-1210_151a431b-f07b-4959-b6fa-c41ff80364c8,0002-1210,HUMAN PRESCRIPTION DRUG,tauvid,NaN,flortaucipir f-18,"INJECTION, SOLUTION",INTRAVENOUS,20200528,NaN,NDA,NDA212123,Eli Lilly and Company,FLORTAUCIPIR F-18,51,mCi/mL,NaN,NaN,N,20221231.0
3,0002-1433_d4ca0797-5a21-4962-bed1-2c4c9b52d78b,0002-1433,HUMAN PRESCRIPTION DRUG,trulicity,NaN,dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20221231.0
4,0002-1434_d4ca0797-5a21-4962-bed1-2c4c9b52d78b,0002-1434,HUMAN PRESCRIPTION DRUG,trulicity,NaN,dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20221231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105042,99207-466_a594be33-33ce-4304-a6fe-65a4eeda3f19,99207-466,HUMAN PRESCRIPTION DRUG,solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Bausch Health US LLC,MINOCYCLINE HYDROCHLORIDE,80,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20221231.0
105043,99207-467_a594be33-33ce-4304-a6fe-65a4eeda3f19,99207-467,HUMAN PRESCRIPTION DRUG,solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Bausch Health US LLC,MINOCYCLINE HYDROCHLORIDE,105,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20221231.0
105044,99207-525_a075b415-7d7c-405a-9645-bdd4c4f92990,99207-525,HUMAN PRESCRIPTION DRUG,vanos,NaN,fluocinonide,CREAM,TOPICAL,20060313,NaN,NDA,NDA021758,"Bausch Health US, LLC",FLUOCINONIDE,1,mg/g,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20221231.0
105045,99207-850_e52ec481-1114-4f09-9be0-311e5eb52fa1,99207-850,HUMAN PRESCRIPTION DRUG,luzu,NaN,luliconazole,CREAM,TOPICAL,20131114,NaN,NDA,NDA204153,"Bausch Health US, LLC",LULICONAZOLE,10,mg/g,"Azole Antifungal [EPC],Azoles [CS],Cytochrome ...",NaN,N,20221231.0


In [35]:
from detectdd.drug_index import DrugIndex

drug_index = DrugIndex.init_with_drugbank()


Loaded drug index with 34681 synonyms


In [36]:
def read_hosp_drugs():
    sql_hosp_drugs_query = """
    SELECT """
    get_authed_client()

In [37]:
def match():
    df = icu_drugs.merge(
        ndc_drug_synonyms.drop_duplicates(subset="NONPROPRIETARYNAME"), left_on= ['label'], right_on=['NONPROPRIETARYNAME'], how='left', indicator=True)[['itemid', 'label', 'category', 'PRODUCTID', 'NONPROPRIETARYNAME']]\
        .merge(ndc_drug_synonyms.drop_duplicates(subset="PROPRIETARYNAME"), left_on=['label'], right_on='PROPRIETARYNAME', how='left', indicator=True)
    print(df.count())
    return df


matched = match()
matched

itemid                              474
label                               474
category                            474
PRODUCTID_x                          71
NONPROPRIETARYNAME_x                 71
PRODUCTID_y                          77
PRODUCTNDC                           77
PRODUCTTYPENAME                      77
PROPRIETARYNAME                      77
PROPRIETARYNAMESUFFIX                 2
NONPROPRIETARYNAME_y                 77
DOSAGEFORMNAME                       77
ROUTENAME                            77
STARTMARKETINGDATE                   77
ENDMARKETINGDATE                      4
MARKETINGCATEGORYNAME                77
APPLICATIONNUMBER                    74
LABELERNAME                          77
SUBSTANCENAME                        76
ACTIVE_NUMERATOR_STRENGTH            76
ACTIVE_INGRED_UNIT                   76
PHARM_CLASSES                        66
DEASCHEDULE                           3
NDC_EXCLUDE_FLAG                     77
LISTING_RECORD_CERTIFIED_THROUGH     73


,itemid,label,category,PRODUCTID_x,NONPROPRIETARYNAME_x,PRODUCTID_y,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,...,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,_merge
0,222139,potassium acetate,Nutrition - Supplements,0409-3294_500bb334-6d33-4bb1-b081-f505423f2872,potassium acetate,0409-3294_500bb334-6d33-4bb1-b081-f505423f2872,0409-3294,HUMAN PRESCRIPTION DRUG,potassium acetate,NaN,...,NDA018896,"Hospira, Inc.",POTASSIUM ACETATE,196.3,mg/mL,"Potassium Compounds [CS],Potassium Salt [EPC],...",NaN,N,20221231.0,both
1,222190,ranitidine,Nutrition - Parenteral,0904-6715_0b8b6ed2-7d1d-4d6d-86f9-0f86fb06b738,ranitidine,0904-6921_2a917fc5-06a6-d0e3-f8d4-39013116a446,0904-6921,HUMAN OTC DRUG,ranitidine,NaN,...,ANDA075294,MAJOR PHARMACEUTICALS,RANITIDINE HYDROCHLORIDE,75,mg/1,NaN,NaN,N,20221231.0,both
2,225801,lipids 20%,Nutrition - Parenteral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,225916,tpn w/ lipids,Nutrition - Parenteral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,225917,tpn without lipids,Nutrition - Parenteral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,221209,peptisorb,Fluids - Other (Not In Use),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
470,221210,suplena,Fluids - Other (Not In Use),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
471,221211,"sodium bicarbonate 1,4%",Fluids - Other (Not In Use),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
472,221212,"saline 0,18%",Fluids - Other (Not In Use),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [38]:

def fuzzy_match(str1, str2):
    return fuzz.token_set_ratio(str1, str2)

def match_dataframe(df1, key1, df2, key2, threshold=90):
    matches = []
    # matches = pd.DataFrame(columns=['index', 'label', 'norm_label', 'match0', 'match1'])
    for i, row in df1.iterrows():
        drug_name = row[key1]
        fast = True
        to_match = df2[key2]
        if fast:
            to_match = to_match.loc[to_match.str.startswith(drug_name[0])] # speed up fuzzy matching by only considering synonyms that start with the same letter

        match = process.extractOne(drug_name, to_match, scorer=fuzzy_match)

        if match[1] >= threshold:
            print(f"Found match with score ({str(match[1])}) : {row[key1]} - {match[0]}")
            matches.append([i, row['itemid'],row[key1], match[0], match[1], match[2]])
        print(i)

    return pd.DataFrame(matches, columns=['index', 'itemid', 'label', 'norm_label', 'score', 'norm_index'])



def fuzzy_merge():
    medications = icu_drugs.loc[(~matched['category'].isin(['Medications']))]
    return match_dataframe(medications, "label", ndc_drug_synonyms, "PROPRIETARYNAME")

fuzzy_matched = fuzzy_merge()
fuzzy_matched

Found match with score (100) : potassium acetate - potassium acetate
0
Found match with score (100) : ranitidine - ranitidine hydrochloride
1
2
3
4
5
6
7
Found match with score (100) : hepatamine - hepatamine
8
9
10
11
Found match with score (97) : potassium phosphate - potassium phosphates
12
Found match with score (100) : sodium acetate - sodium acetate
13
Found match with score (100) : dextrose pn - dextrose
14
15
16
17
18
19
20
21
Found match with score (100) : vanilla scandi shake (mixed) - vanilla
22
23
Found match with score (100) : vanilla lactose free scandi shake (mixed) - vanilla
24
25
26
27
Found match with score (100) : vancomycin - vancomycin hydrochloride
28
Found match with score (100) : acyclovir - acyclovir
29
Found match with score (100) : ambisome - ambisome
30
Found match with score (100) : amikacin - amikacin sulfate
31
Found match with score (100) : ampicillin - ampicillin and sulbactam
32
Found match with score (100) : ampicillin/sulbactam (unasyn) - ampicillin


,index,itemid,label,norm_label,score,norm_index
0,0,222139,potassium acetate,potassium acetate,100,9428
1,1,222190,ranitidine,ranitidine hydrochloride,100,11999
2,8,225921,hepatamine,hepatamine,100,5882
3,12,225925,potassium phosphate,potassium phosphates,97,9571
4,13,225926,sodium acetate,sodium acetate,100,9430
...,...,...,...,...,...,...
103,443,221017,"dextrose 2,5%",dextrose,100,5806
104,468,221208,normosol,normosol-r,100,9594
105,471,221211,"sodium bicarbonate 1,4%",sodium bicarbonate,100,9051
106,472,221212,"saline 0,18%",saline,100,26481


In [39]:
fuzzy_matched

,index,itemid,label,norm_label,score,norm_index
0,0,222139,potassium acetate,potassium acetate,100,9428
1,1,222190,ranitidine,ranitidine hydrochloride,100,11999
2,8,225921,hepatamine,hepatamine,100,5882
3,12,225925,potassium phosphate,potassium phosphates,97,9571
4,13,225926,sodium acetate,sodium acetate,100,9430
...,...,...,...,...,...,...
103,443,221017,"dextrose 2,5%",dextrose,100,5806
104,468,221208,normosol,normosol-r,100,9594
105,471,221211,"sodium bicarbonate 1,4%",sodium bicarbonate,100,9051
106,472,221212,"saline 0,18%",saline,100,26481


In [40]:
m1 = ndc_drug_synonyms['PROPRIETARYNAME'].str.startswith("glyc")
ndc_drug_synonyms.loc[m1]


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
3136,0143-9584_941776c0-fd84-4e62-99c9-55a2a847783d,0143-9584,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,INJECTION,INTRAMUSCULAR; INTRAVENOUS,20150601,NaN,ANDA,ANDA090963,Hikma Pharmaceuticals USA Inc.,GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
3137,0143-9585_941776c0-fd84-4e62-99c9-55a2a847783d,0143-9585,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,INJECTION,INTRAMUSCULAR; INTRAVENOUS,20150601,NaN,ANDA,ANDA090963,Hikma Pharmaceuticals USA Inc.,GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
3138,0143-9586_941776c0-fd84-4e62-99c9-55a2a847783d,0143-9586,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,INJECTION,INTRAMUSCULAR; INTRAVENOUS,20150601,NaN,ANDA,ANDA090963,Hikma Pharmaceuticals USA Inc.,GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
3139,0143-9587_941776c0-fd84-4e62-99c9-55a2a847783d,0143-9587,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,INJECTION,INTRAMUSCULAR; INTRAVENOUS,20150601,NaN,ANDA,ANDA090963,Hikma Pharmaceuticals USA Inc.,GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
3173,0143-9679_96744b85-c7c2-47d5-93fb-4fb55adb54ba,0143-9679,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,INJECTION,INTRAMUSCULAR; INTRAVENOUS,20110921,NaN,ANDA,ANDA090963,Hikma Pharmaceuticals USA Inc.,GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98961,76045-203_d2efb821-f0f9-494d-bed9-71852d2f6fa7,76045-203,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,"INJECTION, SOLUTION",INTRAMUSCULAR; INTRAVENOUS,20180725,NaN,ANDA,ANDA209024,"Fresenius Kabi USA, LLC",GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
98962,76045-206_d2efb821-f0f9-494d-bed9-71852d2f6fa7,76045-206,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,"INJECTION, SOLUTION",INTRAMUSCULAR; INTRAVENOUS,20201030,NaN,ANDA,ANDA209024,"Fresenius Kabi USA, LLC",GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
98963,76045-208_d2efb821-f0f9-494d-bed9-71852d2f6fa7,76045-208,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,"INJECTION, SOLUTION",INTRAMUSCULAR; INTRAVENOUS,20201030,NaN,ANDA,ANDA209024,"Fresenius Kabi USA, LLC",GLYCOPYRROLATE,0.2,mg/mL,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0
102164,79739-7097_392a969d-1e14-4ad8-835c-a47be4983067,79739-7097,HUMAN PRESCRIPTION DRUG,glycopyrrolate,NaN,glycopyrrolate,TABLET,ORAL,20120312,NaN,ANDA,ANDA091522,"LGM Pharma Solutions, LLC",GLYCOPYRROLATE,1.5,mg/1,"Anticholinergic [EPC],Cholinergic Antagonists ...",NaN,N,20221231.0


In [41]:
m1 = ~matched['PRODUCTID_x'].isnull()
m2 = ~matched['PRODUCTID_y'].isnull()
has_product_id = m1 | m2
matched.loc[~has_product_id].groupby('category').count()


,itemid,label,PRODUCTID_x,NONPROPRIETARYNAME_x,PRODUCTID_y,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME_y,...,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,_merge
category,,,,,,,,,,,,,,,,,,,,,
Antibiotics,17,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
Blood Products/Colloids,25,25,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
Fluids - Other (Not In Use),83,83,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,83
Fluids/Intake,36,36,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36
Medications,96,96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,96
Nutrition - Enteral,112,112,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,112
Nutrition - Parenteral,8,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
Nutrition - Supplements,15,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15


In [42]:
matched.loc[(~matched['category'].isin(['Medications'])) & has_product_id]

,itemid,label,category,PRODUCTID_x,NONPROPRIETARYNAME_x,PRODUCTID_y,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,...,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,_merge
0,222139,potassium acetate,Nutrition - Supplements,0409-3294_500bb334-6d33-4bb1-b081-f505423f2872,potassium acetate,0409-3294_500bb334-6d33-4bb1-b081-f505423f2872,0409-3294,HUMAN PRESCRIPTION DRUG,potassium acetate,NaN,...,NDA018896,"Hospira, Inc.",POTASSIUM ACETATE,196.3,mg/mL,"Potassium Compounds [CS],Potassium Salt [EPC],...",NaN,N,20221231.0,both
1,222190,ranitidine,Nutrition - Parenteral,0904-6715_0b8b6ed2-7d1d-4d6d-86f9-0f86fb06b738,ranitidine,0904-6921_2a917fc5-06a6-d0e3-f8d4-39013116a446,0904-6921,HUMAN OTC DRUG,ranitidine,NaN,...,ANDA075294,MAJOR PHARMACEUTICALS,RANITIDINE HYDROCHLORIDE,75,mg/1,NaN,NaN,N,20221231.0,both
8,225921,hepatamine,Nutrition - Supplements,NaN,NaN,0264-9371_c1f4d7ca-8c22-4562-a49b-9d201be140fb,0264-9371,HUMAN PRESCRIPTION DRUG,hepatamine,NaN,...,NDA018676,B. Braun Medical Inc.,ISOLEUCINE; LEUCINE; LYSINE ACETATE; METHIONIN...,.9; 1.1; .61; .1; .1; .45; .066; .84; .77; .6;...,g/100mL; g/100mL; g/100mL; g/100mL; g/100mL; g...,"Amino Acid [EPC],Amino Acids [CS]",NaN,N,20221231.0,both
11,225924,hydrochloric acid,Nutrition - Supplements,0220-3466_8643994f-4e14-4572-e053-2991aa0a5b78,hydrochloric acid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
13,225926,sodium acetate,Nutrition - Supplements,0409-3299_416a7d2a-271d-4b08-a491-1e8a13650b86,sodium acetate,0409-3299_416a7d2a-271d-4b08-a491-1e8a13650b86,0409-3299,HUMAN PRESCRIPTION DRUG,sodium acetate,NaN,...,NDA018893,"Hospira, Inc.",SODIUM ACETATE ANHYDROUS,164,mg/mL,NaN,NaN,N,20221231.0,both
28,225798,vancomycin,Antibiotics,68001-338_c91fb4b4-7b6c-c700-e053-2995a90a6bc2,vancomycin,25021-157_eb393d1e-aff3-4a65-aae8-28418a1aec6c,25021-157,HUMAN PRESCRIPTION DRUG,vancomycin,NaN,...,ANDA200837,Sagent Pharmaceuticals,VANCOMYCIN HYDROCHLORIDE,5,g/100mL,"Glycopeptide Antibacterial [EPC],Glycopeptides...",NaN,N,20221231.0,both
29,225837,acyclovir,Antibiotics,0093-3630_8c3d75a1-fe85-49ca-9bb4-af670f910bf8,acyclovir,0093-3630_8c3d75a1-fe85-49ca-9bb4-af670f910bf8,0093-3630,HUMAN PRESCRIPTION DRUG,acyclovir,NaN,...,NDA021478,"Teva Pharmaceuticals, Inc.",ACYCLOVIR,50,mg/g,"DNA Polymerase Inhibitors [MoA],Herpes Simplex...",NaN,N,20221231.0,both
30,225838,ambisome,Antibiotics,NaN,NaN,0469-3051_8f57e867-909b-4a7c-bd4e-1e9427f5005b,0469-3051,HUMAN PRESCRIPTION DRUG,ambisome,NaN,...,NDA050740,"Astellas Pharma US, Inc.",AMPHOTERICIN B,50,mg/12.5mL,"Lipid-based Polyene Antifungal [EPC],Polyene A...",NaN,N,20221231.0,both
31,225840,amikacin,Antibiotics,71558-590_8b43768c-f7ba-4f33-8509-65a526b0422e,amikacin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
32,225842,ampicillin,Antibiotics,0781-2144_e8f470a8-ec75-4c1e-84d8-766372f61cf3,ampicillin,0409-3718_1e43e05b-8ec3-4ce4-bdfc-f47516ef2efe,0409-3718,HUMAN PRESCRIPTION DRUG,ampicillin,NaN,...,ANDA061395,"Hospira, Inc",AMPICILLIN SODIUM,500,mg/1,"Penicillin-class Antibacterial [EPC],Penicilli...",NaN,N,20221231.0,both


In [43]:
icu_drugs

,itemid,label,abbreviation,category,unitname
0,222139,potassium acetate,Potassium ACEtate,Nutrition - Supplements,mEq
1,222190,ranitidine,Ranitidine,Nutrition - Parenteral,mg
2,225801,lipids 20%,Lipids 20%,Nutrition - Parenteral,mL
3,225916,tpn w/ lipids,TPN w/ Lipids,Nutrition - Parenteral,mL
4,225917,tpn without lipids,TPN without Lipids,Nutrition - Parenteral,mL
...,...,...,...,...,...
469,221209,peptisorb,Peptisorb,Fluids - Other (Not In Use),mL
470,221210,suplena,Suplena,Fluids - Other (Not In Use),mL
471,221211,"sodium bicarbonate 1,4%","NaBic 1,4%",Fluids - Other (Not In Use),mL
472,221212,"saline 0,18%","Saline 0,18%",Fluids - Other (Not In Use),mL


In [44]:
def read_ddinter():
    df = pd.read_csv(ddinter_data_dir / "ddinter_downloads_code_B.csv")
    df.Drug_B = df.Drug_B.fillna("")
    df.Drug_B = df.Drug_B.apply(str.lower)
    df.Drug_B = df.Drug_B.fillna("")
    df.Drug_B = df.Drug_B.apply(str.lower)

    df.Drug_A = df.Drug_A.fillna("")
    df.Drug_A = df.Drug_A.apply(str.lower)
    df.Drug_A = df.Drug_A.fillna("")
    df.Drug_A = df.Drug_A.apply(str.lower)
    return df

ddinter = read_ddinter()
cleaned= ddinter
cleaned.loc[(cleaned['Drug_B'].isin( fuzzy_matched['norm_label']) | cleaned['Drug_A'].isin( fuzzy_matched['norm_label'])) & cleaned.Level.isin( ['Major'])]


,DDInterID_A,Drug_A,DDInterID_B,Drug_B,Level
7,DDInter1683,sodium bicarbonate,DDInter582,dolutegravir,Major
541,DDInter1073,linezolid,DDInter652,epinephrine,Major
1165,DDInter66,amikacin,DDInter1123,mannitol,Major
1175,DDInter68,amiloride,DDInter1495,potassium acetate,Major
1226,DDInter177,benazepril,DDInter1495,potassium acetate,Major
...,...,...,...,...,...
9775,DDInter656,eplerenone,DDInter1495,potassium acetate,Major
9868,DDInter658,eprosartan,DDInter1495,potassium acetate,Major
10075,DDInter743,fluconazole,DDInter542,dicoumarol,Major
10192,DDInter781,fosinopril,DDInter1495,potassium acetate,Major


In [45]:
def query_for_drug_interactions():
    multimap = cleaned.groupby('Drug_A')['Drug_B'].apply(list).to_dict()
    clauses= []
    for key in multimap.keys():
        first_ids = fuzzy_matched.loc[(fuzzy_matched['norm_label'] == key)]['itemid']

        second_ids = fuzzy_matched.loc[(fuzzy_matched['norm_label'].isin(multimap[key]))]['itemid']
        if first_ids.any() & second_ids.any():
            sql = f"(first_ie.itemid IN ({','.join([str(item) for item in first_ids if item])}) AND second_ie.itemid IN ({','.join([str(item) for item in second_ids if item])}))"
            clauses.append(sql)
    print(len(clauses))
    clause = " OR ".join(clauses)
    print (clause)

    icu = "physionet-data.mimiciv_icu"

    sql = f"""SELECT first_ie.subject_id, first_ie.stay_id, first_ie.itemid, second_ie.itemid, MAX(second_ie.starttime) as dose2_time, count(*)
        FROM `{icu}.inputevents` as first_ie
        INNER JOIN `{icu}.inputevents` as second_ie ON first_ie.stay_id = second_ie.stay_id
        WHERE {clause} AND first_ie.amount > 0
            AND second_ie.amount > 0
            AND first_ie.starttime < second_ie.starttime
            AND DATETIME_DIFF(second_ie.starttime, first_ie.starttime, MINUTE) < 300
        GROUP BY first_ie.subject_id, first_ie.stay_id, first_ie.itemid, second_ie.itemid, second_ie.starttime

    """
    print("\n\n",sql)
    mimic_job = get_authed_client().query(sql)
    return mimic_job.to_dataframe()

drug_interactions = query_for_drug_interactions()
drug_interactions

18
(first_ie.itemid IN (225837) AND second_ie.itemid IN (227533,221211)) OR (first_ie.itemid IN (225843) AND second_ie.itemid IN (220988,220989,220990,220991,220992)) OR (first_ie.itemid IN (225844) AND second_ie.itemid IN (227533,221211)) OR (first_ie.itemid IN (225845) AND second_ie.itemid IN (227533,220988,220989,220990,220991,220992,221211)) OR (first_ie.itemid IN (225850) AND second_ie.itemid IN (227533,221211)) OR (first_ie.itemid IN (225855) AND second_ie.itemid IN (227533,221211)) OR (first_ie.itemid IN (225859) AND second_ie.itemid IN (227533,220988,220989,220990,220991,220992,221211)) OR (first_ie.itemid IN (225865) AND second_ie.itemid IN (225926,227533,220988,220989,220990,220991,220992,221211)) OR (first_ie.itemid IN (225866) AND second_ie.itemid IN (227533,220988,220989,220990,220991,220992,221211)) OR (first_ie.itemid IN (225869) AND second_ie.itemid IN (227533,220988,220989,220990,220991,220992,221211)) OR (first_ie.itemid IN (225873) AND second_ie.itemid IN (227533,221

,subject_id,stay_id,itemid,itemid_1,dose2_time,f0_
0,11980812,34036698,225850,227533,2155-12-11 17:30:00,4
1,16337817,37122552,225884,227533,2177-08-04 15:50:00,5
2,14789655,34831657,225884,227533,2152-08-21 23:00:00,4
3,17410025,30645946,225859,227533,2187-08-25 13:50:00,23
4,17410025,30645946,225850,227533,2187-08-25 13:50:00,6
...,...,...,...,...,...,...
6574,15436319,36502055,225869,227533,2125-12-08 09:30:00,15
6575,18756985,30006983,225869,227533,2159-11-16 06:45:00,6
6576,12974332,30720284,225884,227533,2130-12-21 11:55:00,14
6577,12974332,30720284,225869,227533,2130-12-21 11:55:00,3


In [46]:
from detectdd.serializer import Serializer

serializer = Serializer()
serializer.write_cohort(drug_interactions)

Saved cohort


In [47]:
serializer.read_cohort()

Loaded cohort


,subject_id,stay_id,itemid,itemid_1,dose2_time,f0_
0,11980812,34036698,225850,227533,2155-12-11 17:30:00,4
1,16337817,37122552,225884,227533,2177-08-04 15:50:00,5
2,14789655,34831657,225884,227533,2152-08-21 23:00:00,4
3,17410025,30645946,225859,227533,2187-08-25 13:50:00,23
4,17410025,30645946,225850,227533,2187-08-25 13:50:00,6
...,...,...,...,...,...,...
6574,15436319,36502055,225869,227533,2125-12-08 09:30:00,15
6575,18756985,30006983,225869,227533,2159-11-16 06:45:00,6
6576,12974332,30720284,225884,227533,2130-12-21 11:55:00,14
6577,12974332,30720284,225869,227533,2130-12-21 11:55:00,3


In [48]:
drug_interactions

,subject_id,stay_id,itemid,itemid_1,dose2_time,f0_
0,11980812,34036698,225850,227533,2155-12-11 17:30:00,4
1,16337817,37122552,225884,227533,2177-08-04 15:50:00,5
2,14789655,34831657,225884,227533,2152-08-21 23:00:00,4
3,17410025,30645946,225859,227533,2187-08-25 13:50:00,23
4,17410025,30645946,225850,227533,2187-08-25 13:50:00,6
...,...,...,...,...,...,...
6574,15436319,36502055,225869,227533,2125-12-08 09:30:00,15
6575,18756985,30006983,225869,227533,2159-11-16 06:45:00,6
6576,12974332,30720284,225884,227533,2130-12-21 11:55:00,14
6577,12974332,30720284,225869,227533,2130-12-21 11:55:00,3
